### Deploying models as SQL functions
##### Check that we have correct libraries installed

In [2]:
import tensorflow
import keras
import h5py
assert(tensorflow.__version__ == '1.12.0')
assert(keras.__version__ == '2.2.4')
assert(h5py.__version__ == '2.7.0')

/databricks/python/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
 from ._conv import register_converters as _register_converters
Using TensorFlow backend.

##### Download the flower dataset

In [4]:
%sh
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz &>/dev/null
cd flower_photos
ls
pwd

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 1 218M 1 2324k 0 0 4279k 0 0:00:52 --:--:-- 0:00:52 4272k
 56 218M 56 122M 0 0 79.5M 0 0:00:02 0:00:01 0:00:01 79.5M
 84 218M 84 184M 0 0 63.3M 0 0:00:03 0:00:02 0:00:01 63.3M
100 218M 100 218M 0 0 65.4M 0 0:00:03 0:00:03 --:--:-- 65.4M
daisy
dandelion
LICENSE.txt
roses
sunflowers
tulips
/databricks/driver/flower_photos

In [5]:
%sh
rm -rf /databricks/driver/flower_photos/sample_images
cd flower_photos
mkdir sample_images
cp /databricks/driver/flower_photos/daisy/100080576_f52e8ee070_n.jpg sample_images/
cp /databricks/driver/flower_photos/daisy/10140303196_b88d3d6cec.jpg sample_images/
cp /databricks/driver/flower_photos/daisy/10172379554_b296050f82_n.jpg sample_images/

In [6]:
%sh
ls -l /databricks/driver/flower_photos/sample_images

total 180
-rw-r----- 1 root root 26797 May 10 00:50 100080576_f52e8ee070_n.jpg
-rw-r----- 1 root root 117247 May 10 00:50 10140303196_b88d3d6cec.jpg
-rw-r----- 1 root root 36410 May 10 00:50 10172379554_b296050f82_n.jpg

##### Load images into Dataframe

##### Using Spark's ImageSchema

In [9]:
sample_dir = 'file:/databricks/driver/flower_photos/sample_images'

In [10]:
from pyspark.ml.image import ImageSchema
df = ImageSchema.readImages(sample_dir)
df.show()
df.printSchema()

+--------------------+
 image|
+--------------------+
[file:/databricks...|
[file:/databricks...|
[file:/databricks...|
+--------------------+

root
-- image: struct (nullable = true)
 |-- origin: string (nullable = true)
 |-- height: integer (nullable = false)
 |-- width: integer (nullable = false)
 |-- nChannels: integer (nullable = false)
 |-- mode: integer (nullable = false)
 |-- data: binary (nullable = false)

##### Registering SQL User Defined Function

In [12]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

def keras_load_img(fpath):
    from keras.preprocessing.image import load_img, img_to_array
    import numpy as np
    img = load_img(fpath, target_size=(299, 299))
    return img_to_array(img).astype(np.uint8)

registerKerasImageUDF("inceptionV3_udf_with_preprocessing", InceptionV3(weights="imagenet"), keras_load_img)

Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels.h5

 8192/96112376 [..............................] - ETA: 2s
 24576/96112376 [..............................] - ETA: 4:46
 57344/96112376 [..............................] - ETA: 4:05
 122880/96112376 [..............................] - ETA: 2:51
 303104/96112376 [..............................] - ETA: 1:32
 614400/96112376 [..............................] - ETA: 57s 
 1253376/96112376 [..............................] - ETA: 33s
 2531328/96112376 [..............................] - ETA: 19s
 5070848/96112376 [>.............................] - ETA: 10s
 7348224/96112376 [=>............................] - ETA: 7s 
 9003008/96112376 [=>............................] - ETA: 6s
11640832/96112376 [==>...........................] - ETA: 5s
14229504/96112376 [===>..........................] - ETA: 4s
16900096/96112376 [====>.........................] - ETA: 4s
19193856/96112376 [====>.........................] - ETA: 4s
21553152/96112376 [=====>........................] - ETA: 3s
24084480/96112376 [======>.......................] - ETA: 3s
25059328/96112376 [======>.......................] - ETA: 3s
27893760/96112376 [=======>......................] - ETA: 3s
30318592/96112376 [========>.....................] - ETA: 2s
32841728/96112376 [=========>....................] - ETA: 2s
35430400/96112376 [==========>...................] - ETA: 2s
37797888/96112376 [==========>...................] - ETA: 2s
39411712/96112376 [===========>..................] - ETA: 2s
41639936/96112376 [===========>..................] - ETA: 2s
43114496/96112376 [============>.................] - ETA: 2s
45268992/96112376 [=============>................] - ETA: 1s
46997504/96112376 [=============>................] - ETA: 1s
49258496/96112376 [==============>...............] - ETA: 1s
50667520/96112376 [==============>...............] - ETA: 1s
53272576/96112376 [===============>..............] - ETA: 1s
55697408/96112376 [================>.............] - ETA: 1s
58015744/96112376 [=================>............] - ETA: 1s
59777024/96112376 [=================>............] - ETA: 1s
62152704/96112376 [==================>...........] - ETA: 1s
64561152/96112376 [===================>..........] - ETA: 1s
67215360/96112376 [===================>..........] - ETA: 1s
69656576/96112376 [====================>.........] - ETA: 0s
72196096/96112376 [=====================>........] - ETA: 0s
74719232/96112376 [======================>.......] - ETA: 0s
77225984/96112376 [=======================>......] - ETA: 0s
79896576/96112376 [=======================>......] - ETA: 0s
82501632/96112376 [========================>.....] - ETA: 0s
84811776/96112376 [=========================>....] - ETA: 0s
87515136/96112376 [==========================>...] - ETA: 0s
90267648/96112376 [===========================>..] - ETA: 0s
92479488/96112376 [===========================>..] - ETA: 0s
94232576/96112376 [============================>.] - ETA: 0s
96116736/96112376 [==============================] - 3s 0us/step
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
/databricks/python/lib/python3.5/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
 warnings.warn('No training configuration found in save file: '
INFO:tensorflow:Froze 378 variables.
INFO:tensorflow:Converted 378 variables to const ops.
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
 Out[ 7 ]: <sparkdl.graph.builder.GraphFunction at 0x7f6a5acf75c0>

In [13]:
from pyspark.ml.image import ImageSchema

image_df = ImageSchema.readImages(sample_dir)
image_df.registerTempTable("sample_images")
image_df.show()

+--------------------+
 image|
+--------------------+
[file:/databricks...|
[file:/databricks...|
[file:/databricks...|
+--------------------+

In [14]:
%sql 
SELECT inceptionV3_udf_with_preprocessing(image) as predictions from sample_images

predictions


##### Decoding the output

In [16]:
len([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9999994,1.855748e-33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.9097562e-38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.3328338e-37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.2296787e-36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.979189e-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.272725e-15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.4766367e-38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.40103e-34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.4157306e-7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.5694237e-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.97577e-36,0,0,0,0,0,0,0,9.494788e-38,0,0,0,0,0,0,0,0,0,3.4448492e-31,0,0,1.0942032e-37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.8300695e-18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.756805e-27,0,0,1.2987966e-33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

Out[ 9 ]: 1000

In [17]:
import numpy as np
from keras.applications.inception_v3 import preprocess_input, decode_predictions

In [18]:
out = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9999994,1.855748e-33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.9097562e-38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.3328338e-37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.2296787e-36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.979189e-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.272725e-15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.4766367e-38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.40103e-34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.4157306e-7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.5694237e-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.97577e-36,0,0,0,0,0,0,0,9.494788e-38,0,0,0,0,0,0,0,0,0,3.4448492e-31,0,0,1.0942032e-37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.8300695e-18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.756805e-27,0,0,1.2987966e-33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [19]:
out  = np.asarray(out)
print('Predicted:', decode_predictions(out, top=3)[0])

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2016943162039416> in <module> () 
 1 out = np . asarray ( out ) 
 ----> 2 print ( 'Predicted:' , decode_predictions ( out , top = 3 ) [ 0 ] ) 

 /databricks/python/lib/python3.5/site-packages/keras/applications/__init__.py in wrapper (*args, **kwargs) 
 26 kwargs [ 'models' ] = models 
 27 kwargs [ 'utils' ] = utils 
 ---> 28 return base_fun ( * args , ** kwargs ) 
 29 
 30 return wrapper 

 /databricks/python/lib/python3.5/site-packages/keras/applications/inception_v3.py in decode_predictions (*args, **kwargs) 
 14 @ keras_modules_injection 
 15 def decode_predictions ( * args , ** kwargs ) : 
 ---> 16 return inception_v3 . decode_predictions ( * args , ** kwargs ) 
 17 
 18 

 /databricks/python/lib/python3.5/site-packages/keras_applications/imagenet_utils.py in decode_predictions (preds, top, **kwargs) 
 220 'a batch of predictions ' 
 221 '(i.e. a 2D array of shape (samples, 1000)). ' 
 --> 222 'Found array with shape: ' + str(preds.shape))
 223 if CLASS_INDEX is None : 
 224 fpath = keras_utils.get_file(

 ValueError : `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1000,)

In [20]:
out = out.reshape(-1,1000)
out.shape

Out[ 13 ]: (1, 1000)

In [21]:
print('Predicted: ', decode_predictions(out, top=3)[0])

Downloading data from https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

 8192/35363 [=====>........................] - ETA: 0s
24576/35363 [===================>..........] - ETA: 0s
40960/35363 [==================================] - 0s 2us/step
Predicted: [('n01924916', 'flatworm', 0.9999994), ('n03950228', 'pitcher', 5.4157306e-07), ('n03047690', 'clog', 8.272725e-15)]